## Window Theorem (3)

For two separation-identical aircraft $i$ and $j$ where $r_i \le r_j$ and $lt_i \le lt_j$, a *complete-order* can be inferred such that $i$ should precede $j$. Since an aircraft can always be delayed to meet the start of its hard time window, and $t_x \le t'_x$ for all aircraft in corresponding positions of sequences $s$ and $s'$, any time-window violation is no worse in $s$ than in $s'$. Hence, sequencing $i$ before $j$ cannot increase the number or severity of hard time-window violations.


In [7]:
from z3 import *
from itertools import product
from functools import reduce
from colorama import Fore, Style
from tabulate import tabulate

s = Solver()



# introduce some variables
p1, i, p2, j, p3 = Ints('p1 i p2 j p3')
aircraft = [p1, i, p2, j, p3]
s.add(Distinct(*aircraft))

R  = {ac: Real(f"R_{ac}")  for ac in aircraft}
LT = {ac: Real(f"LT_{ac}") for ac in aircraft}
δ   = {(x, y): Real(f"δ_{x}_{y}") for x, y in product(aircraft, aircraft)}



# basic assumptions
for x in aircraft:
    s.add(R[x] > 0, LT[x] > 0)

for (x, y) in product(aircraft, aircraft):
    s.add(δ[x, y] >= 0)
    if x == y:
        s.add(δ[x, y] == 0)



# δ-identical i/j
for x in aircraft:
    s.add(δ[i, x] == δ[j, x])
    s.add(δ[x, i] == δ[x, j])



# helper (symbolic max)
def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs): return reduce(lambda a, b: zmax(a, b), xs)

def compute_T(seq):
    T = {seq[0]: R[seq[0]]}
    for k in range(1, len(seq)):
        preds = seq[:k]
        T[seq[k]] = zmax(R[seq[k]],
                         zmax_list([T[p] + δ[p, seq[k]] for p in preds]))
    return T

def window_violations(T):
    return Sum([If(T[a] > LT[a], 1, 0) for a in aircraft])



# encode s and s'
S       = [p1, i, p2, j, p3]
S_prime = [p1, j, p2, i, p3]

T       = compute_T(S)
T_prime = compute_T(S_prime)

WV      = window_violations(T)
WV_p    = window_violations(T_prime)



# theorem assumptions
s.add(R[i] < R[j])
s.add(LT[i] < LT[j])



# hypothesis
s.add(WV > WV_p)



print("Checking counterexample to Theorem 3 (hard time windows)...")
res = s.check()

def z3_to_float(val):
    if is_rational_value(val):
        return val.numerator_as_long() / val.denominator_as_long()
    if is_algebraic_value(val):
        s = val.approx(20)
        if s.endswith("?"): s = s[:-1]
        return float(s)
    s = str(val)
    if s.endswith("?"): s = s[:-1]
    if "/" in s:
        num, den = s.split("/", 1)
        return float(num) / float(den)
    return float(s)

if res == sat:
    print("\nCounterexample found (would disprove rule):\n")
    m = s.model()

    # Build table rows
    table_data = []
    for ac in aircraft:
        r   = z3_to_float(m.evaluate(R[ac], model_completion=True))
        lt  = z3_to_float(m.evaluate(LT[ac], model_completion=True))
        t   = z3_to_float(m.evaluate(T[ac], model_completion=True))
        tp  = z3_to_float(m.evaluate(T_prime[ac], model_completion=True))
        ok_s  = t   <= lt
        ok_sp = tp  <= lt
        mark_s  = f"{Fore.GREEN}✓{Style.RESET_ALL}" if ok_s  else f"{Fore.RED}✗{Style.RESET_ALL}"
        mark_sp = f"{Fore.GREEN}✓{Style.RESET_ALL}" if ok_sp else f"{Fore.RED}✗{Style.RESET_ALL}"
        table_data.append([str(ac), f"{r:.3f}", f"{lt:.3f}",
                           f"{t:.3f} {mark_s}", f"{tp:.3f} {mark_sp}"])

    # Pretty-print with tabulate
    print(tabulate(
        table_data,
        headers=["Aircraft", "R", "LT", "T(S)", "T(S′)"],
        tablefmt="github",
        floatfmt=".3f",
        stralign="right",
    ))

    # --- Separation matrix -------------------------------------------
    print("\nSeparation Matrix δ(x, y):")
    header = " " * 10 + " ".join([f"{str(b):>10}" for b in aircraft])
    print(header)
    print("-" * len(header))
    for a in aircraft:
        row = [f"{str(a):<10}"]
        for b in aircraft:
            val = z3_to_float(m.evaluate(δ[a, b], model_completion=True))
            row.append(f"{val:10.3f}")
        print("".join(row))

else:
    print("\nUNSAT — Theorem 3 holds: a before b cannot worsen hard-window compliance.")


Checking counterexample to Theorem 3...
sat
Checking counterexample to Theorem 3 (hard time windows)...

Counterexample found (would disprove rule):

|   Aircraft |        R |       LT |       T(S) |      T(S′) |
|------------|----------|----------|------------|------------|
|         p1 | 1670.750 |    1.000 | 1670.750 ✗ | 1670.750 ✗ |
|          i | 1670.250 | 1670.500 | 1670.750 ✗ | 3859.500 ✗ |
|         p2 | 1671.250 | 1671.000 | 1671.250 ✗ | 1671.250 ✗ |
|          j | 1670.500 | 3859.250 | 3859.500 ✗ | 1670.750 ✓ |
|         p3 |    0.250 | 3859.500 | 3859.500 ✓ | 3859.500 ✓ |

Separation Matrix δ(x, y):
                  p1          i         p2          j         p3
----------------------------------------------------------------
p1             0.000     0.000     0.000     0.000     0.250
i              0.000     0.000     0.250     0.000     0.000
p2             0.000  2188.250     0.000  2188.250     0.000
j              0.000     0.000     0.250     0.000     0.000
p3     